In [32]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing

In [33]:
train = pd.read_csv('train.csv')

In [34]:
train.head()

,Id,Target,Duration,Language,Country,Rating,Action,Adventure,Animation,Biography,...,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western,Poster
0,1,0,139,English,France,PG-13,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1.jpg
1,2,0,90,English,USA,R,0,0,0,0,...,0,0,0,0,0,0,1,0,0,2.jpg
2,3,0,103,Mandarin,China,R,1,1,0,0,...,0,0,0,0,0,0,0,0,0,3.jpg
3,4,0,108,Dzongkha,Australia,NaN,0,1,0,0,...,0,0,0,0,0,0,0,0,0,4.jpg
4,5,0,122,English,USA,PG-13,0,0,0,0,...,0,0,0,0,0,0,1,0,0,5.jpg


In [35]:
test = pd.read_csv('test.csv')

In [36]:
test.head()

,Id,Duration,Language,Country,Rating,Action,Adventure,Animation,Biography,Crime,...,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western,Poster
0,3636,89,English,USA,NaN,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,3636.jpg
1,3637,113,English,UK,R,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,3637.jpg
2,3638,97,English,USA,R,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,3638.jpg
3,3639,115,English,USA,R,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,3639.jpg
4,3640,105,English,France,R,1,0,1,0,0,...,0,0,0,0,1,0,1,0,0,3640.jpg


In [37]:
del train['Poster']
del test['Poster']
del train['Id']
id_test = test['Id']
del test['Id']
y = train['Target']
del train['Target']

Бинаризируем признаки

In [38]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [39]:
train.head()

,Duration,Action,Adventure,Animation,Biography,Crime,Documentary,Drama,Family,Fantasy,...,Rating_Not Rated,Rating_PG,Rating_PG-13,Rating_Passed,Rating_R,Rating_TV-14,Rating_TV-G,Rating_TV-MA,Rating_Unrated,Rating_X
0,139,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,90,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,103,1,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,108,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,122,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [40]:
test.head()

,Duration,Action,Adventure,Animation,Biography,Crime,Documentary,Drama,Family,Fantasy,...,Rating_GP,Rating_M,Rating_NC-17,Rating_Not Rated,Rating_PG,Rating_PG-13,Rating_Passed,Rating_R,Rating_Unrated,Rating_X
0,89,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,113,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,97,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,115,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,105,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


После бинаризации признаков оказалось, что некоторые признаки есть в одной таблице ,
но нет во второй.Поэтому добавляем недостающие признаки и помечаем их нулями.

In [41]:
for name in train.columns.values:
    if name not in test.columns.values:
        test[name] = 0
        
for name in test.columns.values:
    if name not in train.columns.values:
        train[name] = 0

In [42]:
train.shape

(3635, 137)

In [43]:
test.shape

(909, 137)

Упорядочим признаки в каждой из таблиц по алфавиту.

In [44]:
train = train.reindex_axis(sorted(train.columns), axis=1)
test = test.reindex_axis(sorted(test.columns), axis=1)

In [45]:
train.head()

,Action,Adventure,Animation,Biography,Country_Afghanistan,Country_Argentina,Country_Aruba,Country_Australia,Country_Bahamas,Country_Belgium,...,Rating_TV-G,Rating_TV-MA,Rating_Unrated,Rating_X,Romance,Sci-Fi,Sport,Thriller,War,Western
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [46]:
test.head()

,Action,Adventure,Animation,Biography,Country_Afghanistan,Country_Argentina,Country_Aruba,Country_Australia,Country_Bahamas,Country_Belgium,...,Rating_TV-G,Rating_TV-MA,Rating_Unrated,Rating_X,Romance,Sci-Fi,Sport,Thriller,War,Western
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0


Используем метод fit, чтобы обучить модель.

In [47]:
scaler = preprocessing.StandardScaler().fit(train.values)
x_train = scaler.transform(train.values) 
x_test = scaler.transform(test.values) 

/home/ilya/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


При помощи логистической регрессии строим линейный классификатор, чтобы определить вероятность принадлежности фильма к комедии.

In [48]:
best_score = 0
best_c = 0
for c in np.logspace(-10, 10, 50):
    clf = LogisticRegression(C=c, penalty='l2', max_iter=100, random_state=42)
    score = cross_val_score(clf, x_train, y.values, 
                            cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring='accuracy').mean()
    print(c, score)
    if best_score < score:
        best_score = score
        best_c = c
print(best_c, best_score)

1e-10 0.763422072488
2.5595479227e-10 0.763422072488
6.5512855686e-10 0.763422072488
1.67683293681e-09 0.763422072488
4.29193426013e-09 0.763422072488
1.09854114199e-08 0.763422072488
2.81176869797e-08 0.763422072488
7.19685673001e-08 0.763422072488
1.84206996933e-07 0.763422072488
4.71486636346e-07 0.763697554582
1.20679264064e-06 0.763697554582
3.08884359648e-06 0.763697554582
7.90604321091e-06 0.764798725096
2.02358964773e-05 0.765349310354
5.17947467923e-05 0.766998415698
0.000132571136559 0.77002341475
0.00033932217719 0.773873345403
0.000868511373751 0.785150687752
0.00222299648253 0.800833845826
0.00568986602902 0.806057785947
0.014563484775 0.806879314467
0.0372759372031 0.804128659678
0.095409547635 0.803304862783
0.244205309455 0.801929724854
0.625055192527 0.802480310111
1.59985871961 0.802204828018
4.09491506238 0.801380274304
10.4811313415 0.801380274304
26.8269579528 0.801930859562
68.6648845004 0.801380652193
175.751062485 0.801380652193
449.843266897 0.801380652193
1151

In [51]:
clf = LogisticRegression(C=0.0145634, penalty='l2', max_iter=1000, random_state=42)

In [52]:
clf.fit(x_train, y.values)

LogisticRegression(C=0.0145634, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=1000,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [113]:
matrix = clf.predict_proba(x_test)

In [118]:
matrix = matrix[:,1]

In [119]:
first = np.array([id_test.values]).T

In [120]:
second = np.array([matrix]).T

In [121]:
print(first.shape, second.shape)

(909, 1) (909, 1)


In [122]:
df = pd.DataFrame(np.concatenate((first, second), axis=1),columns=['id','Probabillity'])

In [123]:
df

,id,Probabillity
0,3636.0,0.385408
1,3637.0,0.110059
2,3638.0,0.661374
3,3639.0,0.050712
4,3640.0,0.206892
5,3641.0,0.645698
6,3642.0,0.098573
7,3643.0,0.569918
8,3644.0,0.441277
9,3645.0,0.291758


In [127]:
pd.DataFrame.to_csv(df, "sumb.csv")